In [24]:
# WANT: Proj SOG

# Model:

# - SOG/GP TY
# - SOG/GP L3-L5
# - Opp Team Allowed SOG/GP TY
# - Opp Team Allowed SOG/GP L3-L5

# Additional:

# - Any Starting Opp D out
# - Starting Opp G out
# - Back to back games
# - Home vs Away
# - Penalties

team = "WPG"
oppTeam = "COL"


In [25]:
# GET TEAM DATA FOR ROSTER PROJ SOG

import pandas as pd

teamUrl = "https://www.hockey-reference.com/teams/" + team + "/"
fullPageData = pd.read_html(teamUrl)
teamData = fullPageData[1]
rosterData = fullPageData[4]
rosterData.columns = rosterData.columns.droplevel()

# Average shots per game
avgS = teamData['S'].iloc[0] / teamData['GP'].iloc[0]

SGPData = rosterData.loc[:,["Player", "GP","S"]]
SGPData['S/GP'] = SGPData['S']/SGPData['GP']
SGPData.drop(SGPData.tail(1).index,inplace=True)
SGPData['S/GP'] = pd.to_numeric(SGPData['S/GP'])
SGPData = SGPData.loc[SGPData['S/GP'] >= 1.5]
SGPData

,Player,GP,S,S/GP
0,Kyle Connor,58,201,3.465517
1,Josh Morrissey,58,117,2.017241
2,Pierre-Luc Dubois,58,166,2.862069
3,Mark Scheifele,58,156,2.689655
4,Blake Wheeler,49,87,1.775510
5,Cole Perfetti,51,90,1.764706
6,Neal Pionk,58,103,1.775862
8,Nikolaj Ehlers,22,71,3.227273
11,Sam Gagner,46,74,1.608696
12,Morgan Barron,46,75,1.630435


In [30]:
# GET OPPONENT DATA
# INCLUDES:
#  -  OPPONENTS SHOTS ALLOWED AVERAGE
#  -  LEAGUE SHOTS ALLOWED AVERAGE

oppUrl = "https://www.hockey-reference.com/teams/" + oppTeam + "/"
oppFullPageData = pd.read_html(oppUrl)
oppTeamData = oppFullPageData[1]
oppRosterData = oppFullPageData[4]
oppRosterData.columns = oppRosterData.columns.droplevel()
oppSA = oppTeamData['SA'].iloc[0] / oppTeamData['GP'].iloc[0]
leagueAvgSA = oppTeamData['SA'].iloc[1] / oppTeamData['GP'].iloc[1]
print(f"Opp SA avg: {oppSA}")
print(f"League SA avg: {leagueAvgSA}")

Opp SA avg: 32.4
League SA avg: 31.344827586206897


In [26]:
# GET AVG SOG OF LAST 5 GAME
# GET PERCENT OF SOG BASED ON TEAM TOTAL AVG SOG
# GET PERCENT OF SOG BASED ON TEAM TOTAL AVG SOG AND LAST 5 GAME AVG SOG

import numpy as np

playerL5AVG = []
playerTeamSPerc = []
playerTeamSPercL5 = []
for index, row in SGPData.iterrows():
    player = row['Player']
    sGP = row['S/GP']
    playerTeamSPerc.append((sGP/avgS)*100)
    url = 'https://www.hockey-reference.com/players/'
    nameSplit = player.split()
    url += nameSplit[1][0]
    url += '/'
    url += nameSplit[1][:5]
    url += nameSplit[0][:2]
    url += '01.html'
    url = url.lower()
    playerL5 = pd.read_html(url)
    playerL5 = playerL5[0]
    playerL5.columns = playerL5.columns.droplevel()
    playerL5.drop(playerL5.tail(1).index,inplace=True)
    playerL5 = pd.to_numeric(playerL5.loc[:,"S"])
    if (playerL5.mean() > 10):
        playerL5AVG.append(np.nan)
        playerTeamSPercL5.append(np.nan)
    else:
        playerL5AVG.append(playerL5.mean())
        playerTeamSPercL5.append((playerL5.mean()/avgS)*100)



SGPData["S/GP L5"] = playerL5AVG
SGPData["S/GP Team %"] = playerTeamSPerc
SGPData["S/GP L5 Team %"] = playerTeamSPercL5
SGPData

,Player,GP,S,S/GP,S/GP L5,S/GP Team %,S/GP L5 Team %
0,Kyle Connor,58,201,3.465517,3.8,11.498856,12.608696
1,Josh Morrissey,58,117,2.017241,NaN,6.693364,NaN
2,Pierre-Luc Dubois,58,166,2.862069,2.6,9.496568,8.627002
3,Mark Scheifele,58,156,2.689655,3.0,8.924485,9.954233
4,Blake Wheeler,49,87,1.775510,1.0,5.891281,3.318078
5,Cole Perfetti,51,90,1.764706,1.4,5.855431,4.645309
6,Neal Pionk,58,103,1.775862,1.8,5.892449,5.972540
8,Nikolaj Ehlers,22,71,3.227273,3.4,10.708342,11.281465
11,Sam Gagner,46,74,1.608696,0.8,5.337777,2.654462
12,Morgan Barron,46,75,1.630435,1.6,5.409909,5.308924


TEMPLATE MATH:

Opponents Allowed SOG Avg

Example : 35.23

Avg SOG

Example: 35.4

Toffoli Avg:

~ 8.7% of the SOG for team

8.7% of 35.23 = Proj 3.065 SOG

Toffoli AVG L5:

~ 9.6%

9.6% of 35.23 = 3.382 SOG

AVG OF ALL SOG AVGS:

3.382 + 3.065 + 3.070 + 3.4 = Aggregate project SOG 3.229

In [28]:
# GETS PROJECTED SOG BASED ON AVG PERCENT OF SOG FROM TEAM TOTAL AND ALLOWED SOG FROM OPPONENT
# GETS SAME AS ABOVE, JUST L5 GAMES

projSOG = []
projSOGL5 = []
for index, row in SGPData.iterrows():
    sPerc = row['S/GP Team %']/100
    sPercL5 = row['S/GP L5 Team %']/100
    projSOG.append(sPerc*oppSA)
    projSOGL5.append(sPercL5*oppSA)
    
SGPData["Proj S"] = projSOG
SGPData["Proj S L5"] = projSOGL5
SGPData

,Player,GP,S,S/GP,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5
0,Kyle Connor,58,201,3.465517,3.8,11.498856,12.608696,3.725629,4.085217
1,Josh Morrissey,58,117,2.017241,NaN,6.693364,NaN,2.168650,NaN
2,Pierre-Luc Dubois,58,166,2.862069,2.6,9.496568,8.627002,3.076888,2.795149
3,Mark Scheifele,58,156,2.689655,3.0,8.924485,9.954233,2.891533,3.225172
4,Blake Wheeler,49,87,1.775510,1.0,5.891281,3.318078,1.908775,1.075057
5,Cole Perfetti,51,90,1.764706,1.4,5.855431,4.645309,1.897160,1.505080
6,Neal Pionk,58,103,1.775862,1.8,5.892449,5.972540,1.909153,1.935103
8,Nikolaj Ehlers,22,71,3.227273,3.4,10.708342,11.281465,3.469503,3.655195
11,Sam Gagner,46,74,1.608696,0.8,5.337777,2.654462,1.729440,0.860046
12,Morgan Barron,46,75,1.630435,1.6,5.409909,5.308924,1.752811,1.720092


In [29]:
# GETS AGGREGATE PROJ SOG BASED ON THE STATS BELOW
#  -  AVG SOG
#  -  AVG SOG L5 GAMES
#  -  PROJ SOG BASED ON OPPONENT AND WHOLE SEASON
#  -  PROJ SOG BASED ON OPPONENT AND AVG L5 GAMES

aggProjSOG = []
for index, row in SGPData.iterrows():
    aggProj = row['S/GP'] + row['S/GP L5'] + row['Proj S'] + row['Proj S L5']
    aggProjSOG.append(aggProj/4)
    
SGPData["Agg Proj S"] = aggProjSOG
SGPData

,Player,GP,S,S/GP,S/GP L5,S/GP Team %,S/GP L5 Team %,Proj S,Proj S L5,Agg Proj S
0,Kyle Connor,58,201,3.465517,3.8,11.498856,12.608696,3.725629,4.085217,3.769091
1,Josh Morrissey,58,117,2.017241,NaN,6.693364,NaN,2.168650,NaN,NaN
2,Pierre-Luc Dubois,58,166,2.862069,2.6,9.496568,8.627002,3.076888,2.795149,2.833526
3,Mark Scheifele,58,156,2.689655,3.0,8.924485,9.954233,2.891533,3.225172,2.951590
4,Blake Wheeler,49,87,1.775510,1.0,5.891281,3.318078,1.908775,1.075057,1.439836
5,Cole Perfetti,51,90,1.764706,1.4,5.855431,4.645309,1.897160,1.505080,1.641736
6,Neal Pionk,58,103,1.775862,1.8,5.892449,5.972540,1.909153,1.935103,1.855030
8,Nikolaj Ehlers,22,71,3.227273,3.4,10.708342,11.281465,3.469503,3.655195,3.437993
11,Sam Gagner,46,74,1.608696,0.8,5.337777,2.654462,1.729440,0.860046,1.249545
12,Morgan Barron,46,75,1.630435,1.6,5.409909,5.308924,1.752811,1.720092,1.675834
